In [126]:
#Dependencies
import http.client
import pandas as pd
import json
import numpy as np
import datetime as dt
import os
import pyodbc
import openpyxl
import sqlalchemy as db

In [127]:
#Set Up Connections
########## EpiInfo ##########
conn = http.client.HTTPSConnection("epi-info-data-service.services.cdc.gov")
conn.set_debuglevel(1)
headers = {
    'authtoken': "Xuv5NBkeYElE0ICxPdTTzw==",
    'accept': "*/*"
    }
########## PDL Database ##########
#Connect to PDL database (SQL SERVER through ODBC)
pdl_engine = db.create_engine("mssql+pyodbc://@Pertussis Lab Database", use_setinputsizes=False)

# #Initiate SQLAlchemy Metadata and table objects - not sure if needed
# metadata = db.MetaData()
# coryne = db.Table(
#     'coryne', 
#     metadata,
#     autoload_with=pdl_engine
# )
# input = db.Table(
#     'Input', 
#     metadata,
#     autoload_with=pdl_engine
# )

In [128]:
#Pull in CCRF data
conn.request("POST", "/api/SurveyData/GetSurveyData?surveyid=0c8b2402-03a4-4599-824c-2bd3a0fe6ab6", headers=headers)

res = conn.getresponse()
r = res.read()
data=json.loads(r.decode("utf-8"))
ccrf_raw=pd.DataFrame(data)
pd.set_option('display.max_columns', 500)

send: b'POST /api/SurveyData/GetSurveyData?surveyid=0c8b2402-03a4-4599-824c-2bd3a0fe6ab6 HTTP/1.1\r\nHost: epi-info-data-service.services.cdc.gov\r\nAccept-Encoding: identity\r\nContent-Length: 0\r\nauthtoken: Xuv5NBkeYElE0ICxPdTTzw==\r\naccept: */*\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Thu, 14 Mar 2024 17:39:21 GMT
header: content-type: text/plain; charset=utf-8
header: server: Kestrel
header: content-length: 684935
header: set-cookie: 9765ae3492289acaddcb56a478ee7257=3e9c15f67d319b1f16d7a0d589e44bc0; path=/; HttpOnly; Secure; SameSite=None


In [184]:
#PDL Data Clean
#Select All PC IDs, add metadata
input_query = "SELECT * FROM INPUT WHERE country ='United States' and ID like 'PC%'"
input_df = pd.read_sql_query(input_query, pdl_engine)
coryne_df = pd.read_sql_table('coryne', pdl_engine)
coryne_ast_df = pd.read_sql_table('coryne_ast', pdl_engine)

In [188]:
#Clean Input
#input_df.columns
input_keep_cols=[
    'ID',
    'acc_num',
    'age_lab',
    'age_units',
    'daterec',
    'datecol',
    'state_lab',
    'spec_type',
    'spec_type_oth',
    'clin_summary',
    'organization'
]
input_final = input_df[input_keep_cols].sort_values('ID',ascending=False)
#input_final.head()

,ID,acc_num,age_lab,age_units,daterec,datecol,state_lab,spec_type,spec_type_oth,clin_summary,organization
1812,PC2389,SHIP240148,49,1,2024-03-13,2024-02-29,WA,6,,None,WA State Department of Health ...
1811,PC2388,SHIP240147,48,1,2024-03-13,2024-02-03,WA,6,,Patient admit Virginia Mason for involuntary t...,WA State Department of Health ...
1810,PC2387,240014000,,9,2024-03-13,2024-02-27,MN,1,,None,"MN PHL Division, Minnesota Department of Healt..."
1809,PC2386,240014001,,9,2024-03-13,2024-02-28,MN,1,,None,"MN PHL Division, Minnesota Department of Healt..."
1808,PC2385,240013389,,9,2024-03-13,2024-02-26,MN,1,,None,"MN PHL Division, Minnesota Department of Healt..."


In [131]:
#Clean Coryne Results
#coryne_df.columns
coryne_keep_cols=[
    'cdcid',
    'toxigenic',
    'cryne_ov_interp',
    'coryne_pcr_interp',
    'biotype'
]
coryne_clean = coryne_df[coryne_df['cdcid'].str.startswith('PC')][coryne_keep_cols].sort_values('cdcid',ascending=False)
#AST
ast_keep_cols = [col for col in coryne_ast_df if (col.endswith('_int')) or (col=='cdcid')]
#Merge Coryne with AST results
coryne_final = pd.merge(left = coryne_clean, right=coryne_ast_df[ast_keep_cols], left_on='cdcid',right_on='cdcid',how='left')
#coryne_clean.head()
ast_keep_cols

['cdcid',
 'coryne_pen_int',
 'coryne_mero_int',
 'coryne_vanc_int',
 'coryne_dapt_int',
 'coryne_azit_int',
 'coryne_eryt_int',
 'coryne_clar_int',
 'coryne_levo_int',
 'coryne_oflo_int',
 'coryne_clin_int',
 'coryne_rifa_int',
 'coryne_amox_int']

In [180]:
#Merge Lab Data
pdl_df= pd.merge(left=input_final, right=coryne_final, left_on='ID',right_on='cdcid',how='left')
pdl_final= pdl_df.drop(['cdcid'],axis=1).rename({'ID':'cdcid'},axis=1)
#Rename Cols
pdl_final = pdl_final.rename(columns={
    "cryne_ov_interp":"CDC_CULT",
    "coryne_pcr_interp":"CDC_PCR",
    "biotype":"CDC_BIOTYPE",
    "toxigenic":"CDC_TOXIGENIC",
    "daterec":"CDC_DATEREC",
    "datecol":"CDC_DATECOL",
    "coryne_pen_int":"CDC_PEN",
    "coryne_mero_int":"CDC_MERO",
    "coryne_vanc_int":"CDC_VANC",
    "coryne_dapt_int":"CDC_DAPT",
    "coryne_azit_int":"CDC_AZIT",
    "coryne_eryt_int":"CDC_ERYT",
    "coryne_clar_int":"CDC_CLAR",
    "coryne_levo_int":"CDC_LEVO",
    "coryne_oflo_int":"CDC_OFLO",
    "coryne_clin_int":"CDC_CLIN",
    "coryne_rifa_int":"CDC_RIFA",
    "coryne_amox_int":"CDC_AMOX"
})

#pdl_final.head()

,cdcid,acc_num,age_lab,age_units,CDC_DATEREC,CDC_DATECOL,state_lab,spec_type,spec_type_oth,clin_summary,organization,CDC_TOXIGENIC,CDC_CULT,CDC_PCR,CDC_BIOTYPE,CDC_PEN,CDC_MERO,CDC_VANC,CDC_DAPT,CDC_AZIT,CDC_ERYT,CDC_CLAR,CDC_LEVO,CDC_OFLO,CDC_CLIN,CDC_RIFA,CDC_AMOX
0,PC2389,SHIP240148,49,1,2024-03-13,2024-02-29,WA,6,,None,WA State Department of Health ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PC2388,SHIP240147,48,1,2024-03-13,2024-02-03,WA,6,,Patient admit Virginia Mason for involuntary t...,WA State Department of Health ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PC2387,240014000,,9,2024-03-13,2024-02-27,MN,1,,None,"MN PHL Division, Minnesota Department of Healt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PC2386,240014001,,9,2024-03-13,2024-02-28,MN,1,,None,"MN PHL Division, Minnesota Department of Healt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PC2385,240013389,,9,2024-03-13,2024-02-26,MN,1,,None,"MN PHL Division, Minnesota Department of Healt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 Clean CCRF - Remove dummy data (GAEXAMPLE), empty responses (state is null) 

In [165]:
ccrf_df=ccrf_raw[(ccrf_raw['RecordID'] != 'GAEXAMPLE') & (ccrf_raw['state'].notnull())]

Clean CCRF - Remove unused columns and rename some columns

In [140]:
ccrf_df = ccrf_df.drop(['_ParentRecordId'],axis=1).rename({
    "_DateUpdated":"EI_DateUpdated",
    "_Status":"EI_Status",
    "GlobalRecordId":"EI_RecordID"
}, axis = 1)

Clean CCRF - Find duplicates with record IDs over length 2, and keep the record with complete status

In [141]:
#Grab Record ids
rids = ccrf_df['RecordID'].value_counts().to_frame().reset_index()
#Select IDs that have length over 2 and have multiple entries
dups = rids[(rids['RecordID'].str.len()>2) & (rids['count']>1)]['RecordID']
#Find Global Record ID of duplicate ID that is listed as not 'Complete'
dups_to_drop = ccrf_df[(ccrf_df['RecordID'].isin(dups)) & (ccrf_df['EI_Status']!='Complete')]['EI_RecordID']
# Drop incomplete dup records   
ccrf_df_final = ccrf_df[~ccrf_df['EI_RecordID'].isin(dups_to_drop)]

In [169]:
#Clean Variable Types
bool_cols = ccrf_df_final.select_dtypes(bool).columns
ccrf_df_final[bool_cols] = ccrf_df_final[bool_cols].astype(int)


Merge CCRF and PDL

In [142]:
#Create copy for merging
merged = ccrf_df_final.copy()
#Intitate list to capture merged isolates
iso_matched = []
#Iterate through sphlid1-sphlid3 and merge on acc_num
for i in range(1,4):
    acc_num = 'acc_num'+str(i)
    sphlid = 'sphlid'+str(i)
    cdcid = 'cdcid'+str(i)
    pdl_merge = pdl_final.add_suffix(str(i)).dropna(subset=[acc_num])
    merged = pd.merge(left=merged, right=pdl_merge,left_on=sphlid,right_on=acc_num,how='left')
    #Collect matched isolates
    [iso_matched.append(cdcid) for cdcid in merged[merged[acc_num].notnull()][cdcid]]

Add unmatched PDL isolates to merged dataframe

In [181]:
#Isolate remaining isolates and add suffix to concat to cdcid1
pdl_remain = pdl_final[~pdl_final['cdcid'].isin(iso_matched)].add_suffix('1')
#Add remaining isolates to merged df
final_df = pd.concat([merged,pdl_remain])



Add System Variables

In [182]:
#Add system variables

#Source
final_df.loc[final_df['cdcid1'].notna(),'system_source']='Lab Only'
final_df.loc[final_df['EI_RecordID'].notna(),'system_source']='CCRF Only'
final_df.loc[(final_df['EI_RecordID'].notna()) & (final_df['cdcid1'].notna()),'system_source']='Combined'
#final_df['source_system'].value_counts()

#Pipeline Run Datetime
final_df['system_datetime']=dt.datetime.now()


In [145]:
#Output CSV
#final_df.to_csv("ccrf.csv",index=False)

Add Data to SQL Table

0      2024-03-14 14:42:04.216930
1      2024-03-14 14:42:04.216930
2      2024-03-14 14:42:04.216930
3      2024-03-14 14:42:04.216930
4      2024-03-14 14:42:04.216930
                  ...            
2384   2024-03-14 14:42:04.216930
2385   2024-03-14 14:42:04.216930
2386   2024-03-14 14:42:04.216930
2387   2024-03-14 14:42:04.216930
2388   2024-03-14 14:42:04.216930
Name: RunDate, Length: 2532, dtype: datetime64[ns]